In [1]:
from pydantic import BaseModel,Field
from typing import Literal
from IPython.display import display, Markdown
from agent.services.llm_service import LLM
from agent.utils.tools import search_web_general,search_web_pdf,get_facts_from_pdf
from agent.agents.planner import PlannerAgent
from agent.core.router import RouterAgent

In [2]:
agent = RouterAgent()

2025-08-09 12:59:30,834 - agent.models.agent_database - INFO - Migrating database schema from version 0 to 1
2025-08-09 12:59:30,835 - agent.models.agent_database - INFO - Performing schema migration from v0 to v1
2025-08-09 12:59:30,835 - agent.models.agent_database - INFO - Schema migration completed: Fresh database initialized


In [3]:
files = ["/app/files/uploads/bryan000/aial201-annual-report-2024-book_v5c19.pdf"]
await agent.activate_conversation("what are major in flight projects for auckland airport, give me a listing with details",files=files)

2025-08-09 12:59:31,342 - agent.core.base - INFO - Your name is Bandit Heeler, your main role is to have a conversation with the user and for complex requests activate agents.Otherwise, you are a fictional character from the show Bluey.
2025-08-09 12:59:31,355 - agent.core.router - INFO - DEBUG: activate_conversation called with files: ['/app/files/uploads/bryan000/aial201-annual-report-2024-book_v5c19.pdf']
2025-08-09 12:59:31,355 - agent.core.router - INFO - DEBUG: message_data prepared: {'message': 'what are major in flight projects for auckland airport, give me a listing with details', 'files': ['/app/files/uploads/bryan000/aial201-annual-report-2024-book_v5c19.pdf']}
2025-08-09 12:59:31,372 - agent.core.router - INFO - DEBUG: handle_message called with files: ['/app/files/uploads/bryan000/aial201-annual-report-2024-book_v5c19.pdf']
2025-08-09 12:59:31,373 - agent.core.router - INFO - DEBUG: files type: <class 'list'>, length: 1
2025-08-09 12:59:31,374 - agent.core.base - INFO - wh

In [4]:
agent.messages[-1]["content"]

"- **Step 1: Extracted Data**  \n  I used the `get_facts_from_pdf` tool to extract detailed information about Auckland Airport's major in-flight projects from the provided PDF document. The extraction yielded structured facts including project names, objectives, timelines, and key stakeholders (see the JSON output).\n\n- **Step 2: Organized the Data**  \n  I organized the extracted facts into a structured list and a table for clarity. The projects identified are:  \n  1. Terminal Integration Programme (TIP)  \n  2. The Stitch (East Terminal Expansion)  \n  3. Airfield Expansion  \n  4. Transport Hub  \n  5. Mānawa Bay Premium Outlet Shopping Centre  \n  6. Te Arikinui Pullman Auckland Airport Hotel  \n  7. Stormwater Infrastructure Upgrades\n\n- **Step 3: Project Details**  \n  - **Terminal Integration Programme (TIP)**  \n    - **Objectives:** Create a vibrant, seamlessly connected airport resilient to climate impacts, with increased capacity and improved passenger experience by integ

In [5]:
display(Markdown(agent.messages[-1]["content"]))

- **Step 1: Extracted Data**  
  I used the `get_facts_from_pdf` tool to extract detailed information about Auckland Airport's major in-flight projects from the provided PDF document. The extraction yielded structured facts including project names, objectives, timelines, and key stakeholders (see the JSON output).

- **Step 2: Organized the Data**  
  I organized the extracted facts into a structured list and a table for clarity. The projects identified are:  
  1. Terminal Integration Programme (TIP)  
  2. The Stitch (East Terminal Expansion)  
  3. Airfield Expansion  
  4. Transport Hub  
  5. Mānawa Bay Premium Outlet Shopping Centre  
  6. Te Arikinui Pullman Auckland Airport Hotel  
  7. Stormwater Infrastructure Upgrades

- **Step 3: Project Details**  
  - **Terminal Integration Programme (TIP)**  
    - **Objectives:** Create a vibrant, seamlessly connected airport resilient to climate impacts, with increased capacity and improved passenger experience by integrating domestic and international terminals under one roof.  
    - **Timeline:** Over 20% complete by end of 2024; expected to open in 2028/2029.  
    - **Stakeholders:** Airlines (e.g., Air New Zealand), Commerce Commission, Programme Director Tim McKenzie, 1,200 workers.  
  - **The Stitch (East Terminal Expansion)**  
    - **Objectives:** Connect the international and domestic terminals with a covered link.  
    - **Timeline:** Part of the broader TIP programme.  
    - **Stakeholders:** Same as TIP.  
  - **Airfield Expansion**  
    - **Objectives:** Add 250,000 sqm of pavement for six new aircraft stands and stormwater upgrades for weather resilience.  
    - **Timeline:** 40% complete; planes expected to park by Q3 2025.  
    - **Stakeholders:** Airfield Programme Director and Project Manager.  
  - **Transport Hub**  
    - **Objectives:** Provide a spacious, covered pick-up/drop-off zone to modernize passenger flow.  
    - **Timeline:** Ground floor opened April 2024; upper levels from October 2024; full completion by end of 2024.  
    - **Stakeholders:** Travellers and transport operators.  
  - **Mānawa Bay Premium Outlet Shopping Centre**  
    - **Objectives:** Establish a purpose-built outlet shopping centre.  
    - **Timeline:** Scheduled to open September 2024.  
    - **Stakeholders:** Retail tenants.  
  - **Te Arikinui Pullman Auckland Airport Hotel**  
    - **Objectives:** Add a five-star hotel near the international terminal.  
    - **Timeline:** Opened December 2023.  
    - **Stakeholders:** Partnership with Tainui Group Holdings.  
  - **Stormwater Infrastructure Upgrades**  
    - **Objectives:** Improve climate resilience with stormwater upgrades, including a biofilter pond.  
    - **Timeline:** 3.5 km of stormwater pipes, 80% installed; complete by March 2025.  
    - **Stakeholders:** Infrastructure programme team.

- **Result:**  
  The comprehensive overview provides a clear picture of Auckland Airport’s ongoing major projects, their objectives, timelines, and stakeholders, based solely on the extracted data from the provided PDF.

In [2]:
llm= LLM()

In [5]:
def search_web_pdf(query: str) -> str:
    """
    Search for PDF documents online using Google Search and extract Q&A pairs from each PDF.
    
    This function searches for PDFs, then extracts facts from each PDF one at a time
    until there are no longer any unanswered questions or the list of PDFs is exhausted.

    Parameters:
    ----------
    query : str
        The search query or topic to find relevant PDF documents for and extract facts from.

    Returns:
    -------
    str
        A JSON string containing comprehensive Q&A pairs extracted from all analysed PDFs,
        along with any remaining unanswered questions.
    """

    class PDFResult(BaseModel):
        description: str = Field(description="Description of the PDF document")
        hyperlink: str = Field(
            description="URL/hyperlink to the PDF document. The link must end in .pdf."
        )

    class PDFSearchResults(BaseModel):
        requested_pdfs: list[PDFResult] = Field(
            [],
            description="List of PDF documents requested. If none found, leave empty.",
        )
        complementary_pdfs: list[PDFResult] = Field(
            [],
            description="If there are non-requested PDF documents that may help address the query, list them here, otherwise leave empty.",
        )

    llm = LLM(caller="tools")
    
    # First, search for PDFs
    response = llm.search_web(
        query=f"Provide hyperlinks to the PDF documents for: {query}\n\n"
        "Can include additional complementary PDFs if noticed."
    )
    pdf_search_results = llm.get_response(
        messages=[{"role": "developer", "content": response}],
        model="gpt-4.1-nano",
        response_format=PDFSearchResults,
    )
    return pdf_search_results

In [6]:
result = search_web_pdf("auckland airport annual report 2024")

2025-08-08 23:18:23,892 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-08-08 23:18:47,262 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent "HTTP/1.1 200 OK"
2025-08-08 23:18:47,265 - google_genai.models - INFO - AFC remote call 1 is done.
2025-08-08 23:18:47,269 - agent.services.llm_service - INFO - LLM Usage: gemini-2.5-pro, Input: 28, Output: 341, Cost: $0.003445
2025-08-08 23:18:50,423 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-08 23:18:50,452 - agent.services.llm_service - INFO - LLM Usage: gpt-4.1-nano-2025-04-14, Input: 433, Output: 205, Cost: $0.000125


In [11]:
result.requested_pdfs[0].hyperlink

'https://www.aucklandairport.co.nz/-/media/aucklandairport/pdf/annual-report-2024.pdf'

In [42]:
import httpx
import requests
# Create a client with session persistence
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
})

# Visit main site first
session.get("https://corporate.aucklandairport.co.nz/")

# Get the PDF
pdf_response = session.get(
    "https://corporate.aucklandairport.co.nz/content/dam/aia/files/corporate/annual-report-2024/aial201-annual-report-2024-book_v5c19.pdf"
)
pdf_data = pdf_response.content

In [43]:
pdf_response

<Response [403]>

In [35]:
pdf_data[:1000]  # Display the first 1000 bytes of the PDF data to verify it's loaded correctly

b'<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">\n<TITLE>302 Moved</TITLE></HEAD><BODY>\n<H1>302 Moved</H1>\nThe document has moved\n<A HREF="https://corporate.aucklandairport.co.nz/content/dam/aia/files/corporate/annual-report-2024/aial201-annual-report-2024-book_v5c19.pdf">here</A>.\r\n</BODY></HTML>\r\n'

In [31]:
facts = get_facts_from_pdf(question="what are major projects that Auckland Airport is undertaking?",pdf_source="https://corporate.aucklandairport.co.nz/content/dam/aia/files/corporate/annual-report-2024/aial201-annual-report-2024-book_v5c19.pdf")

2025-08-09 00:06:07,481 - httpx - INFO - HTTP Request: GET https://corporate.aucklandairport.co.nz/content/dam/aia/files/corporate/annual-report-2024/aial201-annual-report-2024-book_v5c19.pdf "HTTP/1.1 403 Forbidden"
2025-08-09 00:06:07,484 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-08-09 00:06:07,999 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent "HTTP/1.1 400 Bad Request"
2025-08-09 00:06:08,000 - agent.services.llm_service - ERROR - Error in get_response_pdf: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'The document has no pages.', 'status': 'INVALID_ARGUMENT'}}


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'The document has no pages.', 'status': 'INVALID_ARGUMENT'}}

In [8]:
response = llm.search_web(
        query="pdf link to auckland airport annual report 2024"
    )

2025-08-08 23:33:59,363 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-08-08 23:34:12,992 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent "HTTP/1.1 200 OK"
2025-08-08 23:34:12,997 - google_genai.models - INFO - AFC remote call 1 is done.
2025-08-08 23:34:13,003 - agent.services.llm_service - INFO - LLM Usage: gemini-2.5-pro, Input: 14, Output: 193, Cost: $0.001948


In [5]:
response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""## Auckland Airport Sees Strong Rebound in International Travel in 2024

Auckland Airport has released its annual report for the 2024 financial year, highlighting a significant recovery in international passenger numbers and a resulting surge in revenue. The report, covering the twelve months to June 30, 2024, details a year of robust growth as the aviation industry continues to bounce back from the impacts of the pandemic.

The airport experienced a 17% increase in total passenger numbers, reaching 18.5 million. This growth was largely driven by a 29% jump in international passengers, which totaled 10.1 million for the year. Domestic passenger numbers also saw a healthy 5% increase to 8.5 million. This resurgence in travel led to a 43% rise in the airport's revenue, which climbed to $895.5 million.

The strong return

In [10]:
chunks = response.candidates[0].grounding_metadata.grounding_chunks

In [14]:
chunks[1].web.uri

'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFfqgz0aKMQOgwn4VVbmDb4YPY7dX2SC7cIYsVy50jSrBOAi4UHRt7cnI41mU4sZLRKJWssf1bXy7SC56-6s5M3nE0ehHr5SKVUu_ocbvizCPsZqFOSwevdBIYLGyyceZ1Yj8IZFurVlMJ5hun3gZMI3bwzFeZRCdVq_ym0HXJ-k8aVj4gp0jlE0kwUcPgyS4h6hA_B4FUpN16BI2EkJfzgzjqZNOVoSrJ_VxIimRd6dlIR5ADhYknwJQl4ky3vfLI='

In [ ]:
response = requests.get(url, allow_redirects=False, timeout=10)
redirect_url = response.headers.get('Location', '')

In [29]:
import requests

def is_google_redirect_to_pdf(url):
    try:
        response = requests.get(url, allow_redirects=False, timeout=10)

        # Check if it's a redirect
        if response.status_code in [301, 302]:
            redirect_url = response.headers.get('Location', '')
            print(f"Redirect URL: {redirect_url}")
            return redirect_url.lower().endswith('.pdf')

        # Check HTML content for PDF links
        if 'text/html' in response.headers.get('content-type', ''):
            content = response.text.lower()
            return ('.pdf' in content or
                    'application/pdf' in content or
                    'pdf' in content and ('download' in content or 'document' in content))

        return False
    except:
        return False

In [30]:
is_google_redirect_to_pdf(chunks[1].web.uri)

Redirect URL: https://corporate.aucklandairport.co.nz/content/dam/aia/files/corporate/annual-report-2024/aial201-annual-report-2024-book_v5c19.pdf


True

In [12]:
supports = response.candidates[0].grounding_metadata.grounding_supports

In [13]:
supports

[GroundingSupport(
   grounding_chunk_indices=[
     0,
   ],
   segment=Segment(
     end_index=316,
     start_index=219,
     text='The annual results for the twelve months ending June 30, 2024, were announced on August 22, 2024.'
   )
 ),
 GroundingSupport(
   grounding_chunk_indices=[
     1,
   ],
   segment=Segment(
     end_index=609,
     start_index=401,
     text='[https://corporate.aucklandairport.co.nz/-/media/Files/Corporate/Annual-Reports/2024-Annual-Report.ashx](https://corporate.aucklandairport.co.nz/-/media/Files/Corporate/Annual-Reports/2024-Annual-Report.ashx)'
   )
 )]

In [9]:
display(Markdown(response.text))

A direct link to the Auckland Airport Annual Report for the financial year 2024 is available for download. The report, titled "Building for the long haul," details the airport's performance and strategic developments.

The annual results for the twelve months ending June 30, 2024, were announced on August 22, 2024.

You can access the full PDF of the 2024 Annual Report through the following link:

[https://corporate.aucklandairport.co.nz/-/media/Files/Corporate/Annual-Reports/2024-Annual-Report.ashx](https://corporate.aucklandairport.co.nz/-/media/Files/Corporate/Annual-Reports/2024-Annual-Report.ashx)

In [34]:
python_string = """
import numpy as np

# Define the vectors
vector1 = np.array([1, 2, 3])
vector2 = np.array([4, 5, 6])

# Calculate inner product (dot product)
inner_product = np.dot(vector1, vector2)

print(f"Inner product: {inner_product}")
"""

In [35]:
python_string2 = """

# Calculate square root
sqrt_inner_product = np.sqrt(inner_product)
print(f"Square root of inner product: {sqrt_inner_product}")

"""

In [49]:
response =llm.get_response(
    model="gemini-2.5-pro",
    messages=[
        {
            "role": "user",
            "content": f"user request was: calculate the sqrt of inner product of [1,2,3] and [4,5,6]\nFirst step:\n\n```python\n{python_string}\n```\n\nresult is: 56.\n\nSecond step:\n\n```python\n{python_string2}\n```\n\nthe answer is 8.9.\n\n Using the above information only without creating any information, create a comprehensive response to the users request that includes steps taken and their results (where applicable), and format the result in markdown. Return only the markdown answer and nothing else, do not need to repeat the question asked.",
        }
    ],
    # reasoning_effort="none"
).content

2025-08-07 03:22:11,603 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-d57e8caa-0f6c-41f5-b791-e26ed3d08ff4', 'json_data': {'messages': [{'role': 'user', 'content': 'user request was: calculate the sqrt of inner product of [1,2,3] and [4,5,6]\nFirst step:\n\n```python\n\nimport numpy as np\n\n# Define the vectors\nvector1 = np.array([1, 2, 3])\nvector2 = np.array([4, 5, 6])\n\n# Calculate inner product (dot product)\ninner_product = np.dot(vector1, vector2)\n\nprint(f"Inner product: {inner_product}")\n\n```\n\nresult is: 56.\n\nSecond step:\n\n```python\n\n\n# Calculate square root\nsqrt_inner_product = np.sqrt(inner_product)\nprint(f"Square root of inner product: {sqrt_inner_product}")\n\n\n```\n\nthe answer is 8.9.\n\n Using the above information only without creating any information, create a comprehensive response to the users request that includes steps taken and their result

In [50]:
display(Markdown(response))

### Calculation Steps

**Step 1: Calculate the Inner Product**

First, the inner product (dot product) of the two vectors `[1, 2, 3]` and `[4, 5, 6]` is calculated.

**Code:**
```python
import numpy as np

# Define the vectors
vector1 = np.array([1, 2, 3])
vector2 = np.array([4, 5, 6])

# Calculate inner product (dot product)
inner_product = np.dot(vector1, vector2)

print(f"Inner product: {inner_product}")
```

**Result:**
> Inner product: 56

---

**Step 2: Calculate the Square Root**

Next, the square root of the result from the first step (56) is calculated.

**Code:**
```python
# Calculate square root
sqrt_inner_product = np.sqrt(inner_product)
print(f"Square root of inner product: {sqrt_inner_product}")
```

**Result:**
> Square root of inner product: 8.9

---

### Final Answer

The square root of the inner product of `[1, 2, 3]` and `[4, 5, 6]` is **8.9**.

In [3]:
from agent.models.tasks import TaskArtefact
task_result = llm.get_response(
                messages=[{"role": "developer", "content": "calculate square root of 71"}],
                model="sonnet-4",
                temperature=0,
            )

2025-08-04 14:02:49,138 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-aa23785b-9953-4dd0-b97b-f869e77a01ae', 'json_data': {'messages': [{'role': 'user', 'content': 'calculate square root of 71'}], 'model': 'claude-sonnet-4-20250514', 'temperature': 0}}
2025-08-04 14:02:49,140 - openai._base_client - DEBUG - Sending HTTP Request: POST https://api.anthropic.com/v1/chat/completions
2025-08-04 14:02:49,140 - httpcore.connection - DEBUG - connect_tcp.started host='api.anthropic.com' port=443 local_address=None timeout=5.0 socket_options=None
2025-08-04 14:02:49,161 - httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0xffff7dd14440>
2025-08-04 14:02:49,161 - httpcore.connection - DEBUG - start_tls.started ssl_context=<ssl.SSLContext object at 0xffff7e6af1d0> server_hostname='api.anthropic.com' timeout=5.0
2025-08-04 14:02:49,1

In [4]:
display(Markdown(task_result.content))

To calculate the square root of 71, I'll find the approximate value since 71 is not a perfect square.

Let me work through this step by step:

First, I'll identify that √71 is between 8 and 9, since:
- 8² = 64
- 9² = 81

Using a more precise method (like the Newton-Raphson method or long division):

√71 ≈ 8.426

To verify: 8.426² ≈ 70.997, which is very close to 71.

Therefore, **√71 ≈ 8.426** (rounded to three decimal places).

For more precision: √71 ≈ 8.4261497732

In [4]:
print(task_result.model_dump_json(indent=2))

{
  "summary_of_previous_failures": "",
  "thought": "I need to calculate the square root of 71. This is a mathematical calculation that requires code to compute accurately. I'll use Python's math module to calculate the square root and store the result in a variable.",
  "result": "",
  "python_code": "import math\n\nsquare_root_of_71 = math.sqrt(71)\nprint(f\"The square root of 71 is: {square_root_of_71}\")",
  "output_variables": [
    {
      "name": "square_root_of_71",
      "is_image": false
    }
  ],
  "is_malicious": false
}


In [5]:
response = llm.get_response(messages=[{"role":"developer","content":"tell me a joke"},

{"role":"assistant","content":"why did the chicken cross the road?"},

{"role":"user","content":"this is a really old joke, tell me something else"}], model="gemini-2.5-pro").content
print(response)

You're right, my apologies. That one's had its day.

How about this:

I'm reading a book on anti-gravity. It's impossible to put down


In [ ]:
# View all router agent messages from SQLite database
import pandas as pd
import sqlite3
from pathlib import Path

# Database path (adjust if needed)
db_path = Path("../db/agent_messages.db")

# Check if database exists
if db_path.exists():
    # Connect to SQLite database
    conn = sqlite3.connect(db_path)
    
    # Query all router messages
    query = """
    SELECT 
        rm.id,
        rm.conversation_id,
        rm.role,
        rm.content,
        rm.created_at,
        c.created_at as conversation_created_at
    FROM router_messages rm
    LEFT JOIN conversations c ON rm.conversation_id = c.id
    ORDER BY rm.created_at DESC
    """
    
    # Execute query and create DataFrame
    df = pd.read_sql_query(query, conn)
    
    # Close connection
    conn.close()
    
else:
    print(f"Database not found at {db_path}")
    print("Alternative paths to check:")
    

In [3]:
len("im bored and i want to watch some tv")

36

In [4]:
df

,id,conversation_id,role,content,created_at,conversation_created_at
0,60,default,system,"Your name is Bandit Heeler, your main role is ...",2025-07-31 12:51:18.930629,2025-07-29 12:17:31.124339
1,59,default,system,"Your name is Bandit Heeler, your main role is ...",2025-07-31 12:47:21.148905,2025-07-29 12:17:31.124339
2,58,default,system,"Your name is Bandit Heeler, your main role is ...",2025-07-31 12:40:41.884868,2025-07-29 12:17:31.124339
3,57,default,system,"Your name is Bandit Heeler, your main role is ...",2025-07-31 12:32:42.157878,2025-07-29 12:17:31.124339
4,56,default,system,"Your name is Bandit Heeler, your main role is ...",2025-07-31 12:25:26.346056,2025-07-29 12:17:31.124339
5,55,default,system,"Your name is Bandit Heeler, your main role is ...",2025-07-31 12:24:53.315421,2025-07-29 12:17:31.124339
6,54,default,system,"Your name is Bandit Heeler, your main role is ...",2025-07-31 12:24:16.237427,2025-07-29 12:17:31.124339
7,53,default,assistant,Looks like that message might have gotten cut ...,2025-07-31 12:12:48.108680,2025-07-29 12:17:31.124339
8,52,default,user,t,2025-07-31 12:12:46.825543,2025-07-29 12:17:31.124339
9,51,default,assistant,Sure thing! Here's a fun one:\n\nWhy did the b...,2025-07-31 12:01:22.868934,2025-07-29 12:17:31.124339
